In [ ]:
!pip install numpy
!pip install pandas-ta
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 14.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: pandas
    Found existing insta

KeyboardInterrupt: 

In [ ]:
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import pandas_ta as ta
from pandas import read_csv
from pandas.core.frame import DataFrame
import os
import numpy as np
from numpy import split, array, dstack
import math
from math import sqrt
import seaborn as sns
import csv
from scipy.stats import zscore
from datetime import datetime
import pickle
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score

### Company data feature engineering functions

In [ ]:
# fill missing values with a value at the same time one day ago
def fill_missing(values):
  return pd.DataFrame(values).fillna(method='ffill').values

def normalized_df(df):
    normalized_df=(df-df.mean())/df.std()
    return normalized_df

def zlema(src, length):
   lag = round((length - 1) / 2)
   ema_data = src + (src - src[lag])
   zl= ta.ema(ema_data,length = 5)
   return zl


In [ ]:
def get_company_df_sa(dataset,company_name,start_date):
  company = dataset[(dataset['trading_name '] == company_name)]
  new_date_range = pd.date_range(start=start_date, end="2020-03-05", freq="D")
  company = company.reindex(new_date_range, fill_value=np.NaN)
  company = company.drop(["symbol",'name','trading_name ','sectoer'], axis='columns')
  company.replace('?', np.NaN, inplace=True)
  company = company.astype('float32')
  fill_missing(company.values)
  values = company.values
  company['zg'],company['sg'],company['xg'],company['atr'] = ta.aberration(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['lower'],company['mid'],company['upper'] = ta.accbands(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['ad'] = ta.ad(high=company['high'],low= company['low'],close=company['close'],volume=company['volume_traded '],length=5)
  company['adosc'] = ta.adosc(high=company['high'],low= company['low'],volume=company['volume_traded '],close=company['close'],fast=1,slow=5)
  company['adx'],company['dmp'],company['dmn'] = ta.adx(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['alma'] = ta.alma(company['close'],length=5)
  company['amat1'], company['amat2']= ta.amat(company['close'],length=5)
  company['ao'] = ta.ao(high=company['high'],low= company['low'],close=company['close'],fast=1,slow=5)
  company['aobv1'],company['aobv2'],company['aobv3'],company['aobv4'],company['aobv5'],company['aobv6'],company['aobv7'] = ta.aobv(company['close'],company['volume_traded '])
  company['apo'] = ta.apo(company['close'],fast=1,slow=5)
  company['aroon_up'],company['aroon_down'],company['aroon_osc'] = ta.aroon(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['atr']= ta.atr(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['bbands_lower'],company['bbands_mid'],company['bbands_upper'],company['bbands_bandwidth'],company['bbands_percent'] = ta.bbands(company['close'],length=5)
  company['bias'] = ta.bias(company['close'],length=5)
  company['bop']= ta.bop(open_=company['open'],high=company['high'],low= company['low'],close=company['close'],length=5)
  company['ar'],company['br']= ta.brar(open_ = company['open'],high=company['high'],low= company['low'],close=company['close'],length=5)
  company['cci']= ta.cci(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['cfo'] = ta.cfo(company['close'],length=5)
  company['cg'] = ta.cg(company['close'],length=5)
  company['chop']= ta.chop(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['cksp_long'], company['cksp_short']= ta.cksp(high=company['high'],low= company['low'],close=company['close'])
  company['cmf']= ta.cmf(high=company['high'],low= company['low'],close=company['close'],volume = company['volume_traded '],open = company['open'],length=5)
  company['cmo'] = ta.cmo(company['close'],length=5)
  company['coppock'] = ta.coppock(company['close'],length=5,fast=6,slow=11)
  company['cti'] = ta.cti(company['close'],length=5)
  company['decay'] = ta.decay(company['close'],length=5)
  company['decreasing'] = ta.decreasing(company['close'],length=5)
  company['dema'] = ta.dema(company['close'],length=5)
  company['dmp'],company['dmn']= ta.dm(high=company['high'],low= company['low'],length=5)
  company['doncian_lower'],company['doncian_mid'],company['doncian_upper']= ta.donchian(high=company['high'],low= company['low'],lower_length=5,upper_length=5)
  company['dpo'] = ta.dpo(company['close'],length=5)
  company['ebsw'] = ta.ebsw(company['close'],length=5)
  company['efi'] = ta.efi(company['close'],company['volume_traded '],length=5)
  company['ema'] = ta.ema(company['close'],length=5)
  company['entropy'] = ta.entropy(company['close'],length=5)
  company['eom']= ta.eom(high=company['high'],low= company['low'],close=company['close'],volume = company['volume_traded '],length=5)
  company['er'] = ta.er(company['close'],length=5)
  company['eri_bull'],company['eri_bear'] = ta.eri(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['fisher1'],company['fisher2'] = ta.fisher(high=company['high'],low= company['low'],length=5)
  company['fwma'] = ta.fwma(company['close'],length=5)
  company['hilo'],company['hilol'],company['hilos']=ta.hilo(close=company['close'],high=company['high'],low= company['low'],high_length=5,low_length=5)
  company['hma'] = ta.hma(company['close'],length=5)
  company['hl2'] = ta.hl2(company['high'],company['low'])
  company['hlc3'] = ta.hlc3(company['high'],company['low'],company['close'])
  company['hwm'],company['hwu'],company['hwl'] = ta.hwc(company['close'])
  company['hwma '] = ta.dema(company['close'])
  company['increasing'] = ta.increasing(company['close'],length=5)
  company['inertia'] = ta.inertia(high=company['high'],low= company['low'],close=company['close'],open = company['open'],rvi_length=5,length=5)
  company['jma'] = ta.jma(company['close'],length=5)
  company['kama'] = ta.kama(company['close'],length=5)
  company['kcl'],company['kcb'],company['kcu']= ta.kc(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['kdj1'],company['kdj2'],company['kdj3'] = ta.kdj(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['kurtosis'] = ta.kurtosis(company['close'],length=5)
  company['kvo'],company['kvo_s'] = ta.kvo(high=company['high'],low= company['low'],close=company['close'],volume = company['volume_traded '])
  company['linreg'] = ta.linreg(company['close'],length=5)
  company['log_return'] = ta.log_return(company['close'],length=5)
  company['macd'],company['macd_hist'],company['macd_sig'] = ta.macd(company['close'])
  company['mad'] = ta.mad(company['close'],length=5)
  company['massi'] = ta.massi(high=company['high'],low= company['low'],fast=1,slow=5)
  company['median'] = ta.median(company['close'],length=5)
  company['mfi'] = ta.mfi(high=company['high'],low= company['low'],close=company['close'],volume=company['volume_traded '],length=5)
  company['midpoint'] = ta.midpoint(company['close'],length=5)
  company['midprice'] = ta.midprice(high=company['high'],low= company['low'],length=5)
  company['mom'] = ta.mom(company['close'],length=5)
  company['natr'] = ta.natr(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['nvi'] = ta.nvi(company['close'],company['volume_traded '],length=5)
  company['obv'] = ta.obv(company['close'],company['volume_traded '])
  company['pdist'] = ta.pdist(open_=company['open'],high=company['high'],low= company['low'],close=company['close'])
  company['percent_return'] = ta.percent_return(company['close'],length=5)
  company['pgo'] = ta.pgo(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['psl'] = ta.psl(company['close'],company['open'],length=5)
  company['pvi'] = ta.pvi(company['close'],company['volume_traded '],length=5)
  company['pvol'] = ta.pvol(company['close'],company['volume_traded '])
  company['pvt'] = ta.pvt(company['close'],company['volume_traded '])
  company['pwma'] = ta.pwma(company['close'],length=5)
  company['qqe'],company['rsi_ma'] ,company['qqel'] ,company['qqes']  = ta.qqe(company['close'],length=5)
  company['qstick'] = ta.qstick(company['open'],company['close'],length=5)
  company['quantile'] = ta.quantile(company['close'],length=5)
  company['rma'] = ta.rma(company['close'],length=5)
  company['roc'] = ta.roc(company['close'],length=5)
  company['rsi'] = ta.rsi(company['close'],length=5)
  company['rsx'] = ta.rsx(company['close'],length=5)
  company['rvgi1'],company['rvgi2'] = ta.rvgi(open_=company['open'],high=company['high'],low= company['low'],close=company['close'],length=5)
  #company['rvil'],company['rvib'],company['rviu'],company['rvi1'],company['rvi2'],company['rvi3'],company['rvi4'],company['rvi5'],company['rvi6'],company['rvi7'],company['rvi8'],company['rvi9'],company['rvi10'],company['rvi11'] = ta.rvi(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['sinwma'] = ta.sinwma(company['close'],length=5)
  company['skew'] = ta.skew(company['close'],length=5)
  company['slope'] = ta.slope(company['close'],length=5)
  company['sma'] = ta.sma(company['close'],length=5)
  company['smi'],company['smi_s'],company['smi_osc'] = ta.smi(company['close'])
  company['ssf'] = ta.ssf(company['close'],length=5)
  company['stc'],company['stc_macd'],company['stc_stoch'] = ta.stc(company['close'],tclength=5)
  company['stdev'] = ta.stdev(company['close'],length=5)
  company['stoch_k'],company['stoch_d'] = ta.momentum.stoch(high=company['high'],low= company['low'],close=company['close'])
  company['stochrsi_k'],company['stochrsi_d']= ta.stochrsi(company['close'],length=5)
  company['swma'] = ta.swma(company['close'],length=5)
  company['t3'] = ta.t3(company['close'],length=5)
  company['tema'] = ta.tema(company['close'],length=5)
  company['trima'] = ta.trima(company['close'],length=5)
  company['trix1'],company['trix2']  = ta.trix(company['close'],length=5)
  company['true_range'] = ta.true_range(high=company['high'],low= company['low'],close=company['close'])
  company['tema'] = ta.tema(company['close'],length=5)
  company['ui'] = ta.ui(high=company['high'],close =company['close'],length=5)
  company['uo'] = ta.uo(company['high'],company['low'],company['close'],fast=1,medium=3,slow=5)
  company['variance'] = ta.variance(company['close'],length=5)
  company['vhf'] = ta.vhf(company['close'],length=5)
  company['vidya'] = ta.vidya(company['close'],length=5)
  company['vortex1'],company['vortex2'] = ta.vortex(company['high'],company['low'],company['close'],length=5)
  company['vp1'],company['vp2'],company['vp3'],company['vp4'],company['vp5'],company['vp6'] = ta.vp(company['close'],company['volume_traded '])
  company['vwap'] = ta.vwap(high=company['high'],low= company['low'],close=company['close'],volume=company['volume_traded '])
  company['vmwa'] = ta.vwma(volume=company['volume_traded '],close=company['close'],length=5)
  company['wcp'] = ta.wcp(high=company['high'],low= company['low'],close=company['close'])
  company['willr'] = ta.willr(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['wma'] = ta.wma(company['close'],length=5)
  company['zlma'] = ta.zlma(company['close'],length=5)
  company['zscore'] = ta.zscore(company['close'],length=5)
  #company['zlema'] = zlema(company['close'],length=5)
  company = company.fillna(0)
  #selected features based on corrlation sheet
  selected_ti = ['open','high','low','close','alma','decay','dema','ema','fwma','hma','hl2','hlc3','hwma ','jma','kama','linreg','median','midpoint','midprice','pwma','quantile','rma','sinwma','sma','ssf','swma','t3','tema','trima','vwap','vmwa','wcp','wma','zlma']
  new_company_df =  company[selected_ti]
  return new_company_df, selected_ti

In [ ]:
def get_company_df_us(company,company_name):
  new_date_range = pd.date_range(start="31-12-2001", end="2020-03-05", freq="D")
  company = company.reindex(new_date_range, fill_value=np.NaN)
  company.replace('?', np.NaN, inplace=True)
  company = company.astype('float32')
  company[:] = fill_missing(company.values)
  company['zg'],company['sg'],company['xg'],company['atr'] = ta.aberration(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['lower'],company['mid'],company['upper'] = ta.accbands(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['ad'] = ta.ad(high=company['High'],low= company['Low'],close=company['Close'],volume=company['Volume'],length=5)
  company['adosc'] = ta.adosc(high=company['High'],low= company['Low'],volume=company['Volume'],close=company['Close'],fast=1,slow=5)
  company['adx'],company['dmp'],company['dmn'] = ta.adx(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['alma'] = ta.alma(company['Close'],length=5)
  company['amat1'], company['amat2']= ta.amat(company['Close'],length=5)
  company['ao'] = ta.ao(high=company['High'],low= company['Low'],close=company['Close'],fast=1,slow=5)
  company['aobv1'],company['aobv2'],company['aobv3'],company['aobv4'],company['aobv5'],company['aobv6'],company['aobv7'] = ta.aobv(company['Close'],company['Volume'])
  company['apo'] = ta.apo(company['Close'],fast=1,slow=5)
  company['aroon_up'],company['aroon_down'],company['aroon_osc'] = ta.aroon(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['atr']= ta.atr(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['bbands_lower'],company['bbands_mid'],company['bbands_upper'],company['bbands_bandwidth'],company['bbands_percent'] = ta.bbands(company['Close'],length=5)
  company['bias'] = ta.bias(company['Close'],length=5)
  company['bop']= ta.bop(open_=company['Open'],high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['ar'],company['br']= ta.brar(open_ = company['Open'],high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['cci']= ta.cci(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['cfo'] = ta.cfo(company['Close'],length=5)
  company['cg'] = ta.cg(company['Close'],length=5)
  company['chop']= ta.chop(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['cksp_long'], company['cksp_short']= ta.cksp(high=company['High'],low= company['Low'],close=company['Close'])
  company['cmf']= ta.cmf(high=company['High'],low= company['Low'],close=company['Close'],volume = company['Volume'],open = company['Open'],length=5)
  company['cmo'] = ta.cmo(company['Close'],length=5)
  company['coppock'] = ta.coppock(company['Close'],length=5,fast=6,slow=11)
  company['cti'] = ta.cti(company['Close'],length=5)
  company['decay'] = ta.decay(company['Close'],length=5)
  company['decreasing'] = ta.decreasing(company['Close'],length=5)
  company['dema'] = ta.dema(company['Close'],length=5)
  company['dmp'],company['dmn']= ta.dm(high=company['High'],low= company['Low'],length=5)
  company['doncian_lower'],company['doncian_mid'],company['doncian_upper']= ta.donchian(high=company['High'],low= company['Low'],lower_length=5,upper_length=5)
  company['dpo'] = ta.dpo(company['Close'],length=5)
  company['ebsw'] = ta.ebsw(company['Close'],length=5)
  company['efi'] = ta.efi(company['Close'],company['Volume'],length=5)
  company['ema'] = ta.ema(company['Close'],length=5)
  company['entropy'] = ta.entropy(company['Close'],length=5)
  company['eom']= ta.eom(high=company['High'],low= company['Low'],close=company['Close'],volume = company['Volume'],length=5)
  company['er'] = ta.er(company['Close'],length=5)
  company['eri_bull'],company['eri_bear'] = ta.eri(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['fisher1'],company['fisher2'] = ta.fisher(high=company['High'],low= company['Low'],length=5)
  company['fwma'] = ta.fwma(company['Close'],length=5)
  company['hilo'],company['hilol'],company['hilos']=ta.hilo(close=company['Close'],high=company['High'],low= company['Low'],high_length=5,low_length=5)
  company['hma'] = ta.hma(company['Close'],length=5)
  company['hl2'] = ta.hl2(company['High'],company['Low'])
  company['hlc3'] = ta.hlc3(company['High'],company['Low'],company['Close'])
  company['hwm'],company['hwu'],company['hwl'] = ta.hwc(company['Close'])
  company['hwma '] = ta.dema(company['Close'])
  company['increasing'] = ta.increasing(company['Close'],length=5)
  company['inertia'] = ta.inertia(high=company['High'],low= company['Low'],close=company['Close'],open = company['Open'],rvi_length=5,length=5)
  company['jma'] = ta.jma(company['Close'],length=5)
  company['kama'] = ta.kama(company['Close'],length=5)
  company['kcl'],company['kcb'],company['kcu']= ta.kc(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['kdj1'],company['kdj2'],company['kdj3'] = ta.kdj(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['kurtosis'] = ta.kurtosis(company['Close'],length=5)
  company['kvo'],company['kvo_s'] = ta.kvo(high=company['High'],low= company['Low'],close=company['Close'],volume = company['Volume'])
  company['linreg'] = ta.linreg(company['Close'],length=5)
  company['log_return'] = ta.log_return(company['Close'],length=5)
  company['macd'],company['macd_hist'],company['macd_sig'] = ta.macd(company['Close'])
  company['mad'] = ta.mad(company['Close'],length=5)
  company['massi'] = ta.massi(high=company['High'],low= company['Low'],fast=1,slow=5)
  company['median'] = ta.median(company['Close'],length=5)
  company['mfi'] = ta.mfi(high=company['High'],low= company['Low'],close=company['Close'],volume=company['Volume'],length=5)
  company['midpoint'] = ta.midpoint(company['Close'],length=5)
  company['midprice'] = ta.midprice(high=company['High'],low= company['Low'],length=5)
  company['mom'] = ta.mom(company['Close'],length=5)
  company['natr'] = ta.natr(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['nvi'] = ta.nvi(company['Close'],company['Volume'],length=5)
  company['obv'] = ta.obv(company['Close'],company['Volume'])
  company['pdist'] = ta.pdist(open_=company['Open'],high=company['High'],low= company['Low'],close=company['Close'])
  company['percent_return'] = ta.percent_return(company['Close'],length=5)
  company['pgo'] = ta.pgo(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['psl'] = ta.psl(company['Close'],company['Open'],length=5)
  company['pvi'] = ta.pvi(company['Close'],company['Volume'],length=5)
  company['pvol'] = ta.pvol(company['Close'],company['Volume'])
  company['pvt'] = ta.pvt(company['Close'],company['Volume'])
  company['pwma'] = ta.pwma(company['Close'],length=5)
  company['qqe'],company['rsi_ma'] ,company['qqel'] ,company['qqes']  = ta.qqe(company['Close'],length=5)
  company['qstick'] = ta.qstick(company['Open'],company['Close'],length=5)
  company['quantile'] = ta.quantile(company['Close'],length=5)
  company['rma'] = ta.rma(company['Close'],length=5)
  company['roc'] = ta.roc(company['Close'],length=5)
  company['rsi'] = ta.rsi(company['Close'],length=5)
  company['rsx'] = ta.rsx(company['Close'],length=5)
  company['rvgi1'],company['rvgi2'] = ta.rvgi(open_=company['Open'],high=company['High'],low= company['Low'],close=company['Close'],length=5)
  #company['rvil'],company['rvib'],company['rviu'],company['rvi1'],company['rvi2'],company['rvi3'],company['rvi4'],company['rvi5'],company['rvi6'],company['rvi7'],company['rvi8'],company['rvi9'],company['rvi10'],company['rvi11'] = ta.rvi(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['sinwma'] = ta.sinwma(company['Close'],length=5)
  company['skew'] = ta.skew(company['Close'],length=5)
  company['slope'] = ta.slope(company['Close'],length=5)
  company['sma'] = ta.sma(company['Close'],length=5)
  company['smi'],company['smi_s'],company['smi_osc'] = ta.smi(company['Close'])
  company['ssf'] = ta.ssf(company['Close'],length=5)
  company['stc'],company['stc_macd'],company['stc_stoch'] = ta.stc(company['Close'],tclength=5)
  company['stdev'] = ta.stdev(company['Close'],length=5)
  company['stoch_k'],company['stoch_d'] = ta.momentum.stoch(high=company['High'],low= company['Low'],close=company['Close'])
  company['stochrsi_k'],company['stochrsi_d']= ta.stochrsi(company['Close'],length=5)
  company['swma'] = ta.swma(company['Close'],length=5)
  company['t3'] = ta.t3(company['Close'],length=5)
  company['tema'] = ta.tema(company['Close'],length=5)
  company['trima'] = ta.trima(company['Close'],length=5)
  company['trix1'],company['trix2']  = ta.trix(company['Close'],length=5)
  company['true_range'] = ta.true_range(high=company['High'],low= company['Low'],close=company['Close'])
  company['tema'] = ta.tema(company['Close'],length=5)
  company['ui'] = ta.ui(high=company['High'],close =company['Close'],length=5)
  company['uo'] = ta.uo(company['High'],company['Low'],company['Close'],fast=1,medium=3,slow=5)
  company['variance'] = ta.variance(company['Close'],length=5)
  company['vhf'] = ta.vhf(company['Close'],length=5)
  company['vidya'] = ta.vidya(company['Close'],length=5)
  company['vortex1'],company['vortex2'] = ta.vortex(company['High'],company['Low'],company['Close'],length=5)
  company['vp1'],company['vp2'],company['vp3'],company['vp4'],company['vp5'],company['vp6'] = ta.vp(company['Close'],company['Volume'])
  company['vwap'] = ta.vwap(high=company['High'],low= company['Low'],close=company['Close'],volume=company['Volume'])
  company['vmwa'] = ta.vwma(volume=company['Volume'],close=company['Close'],length=5)
  company['wcp'] = ta.wcp(high=company['High'],low= company['Low'],close=company['Close'])
  company['willr'] = ta.willr(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['wma'] = ta.wma(company['Close'],length=5)
  company['zlma'] = ta.zlma(company['Close'],length=5)
  company['zscore'] = ta.zscore(company['Close'],length=5)
  #company['zlema'] = zlema(company['Close'],length=5)
  company = company.fillna(0)
  #selected features based on corrlation sheet
  selected_us_ti = ['Adjusted Close',
  'Close',
  'High',
  'Low',
  'Open',
  'alma',
  'decay',
  'dema',
  'ema',
  'fwma',
  'hl2',
  'hlc3',
  'hma',
  'hwma ',
  'linreg',
  'median',
  'midpoint',
  'midprice',
  'pvi',
  'pwma',
  'quantile',
  'rma',
  'sinwma',
  'sma',
  'swma',
  't3',
  'tema',
  'trima',
  'vmwa',
  'vwap',
  'wcp',
  'wma',
  'zlma']
  new_company_df =  company[selected_us_ti]
  return new_company_df, selected_us_ti

## Excution

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/KFUPM/Graduation/data/US/BAC_selected_features.csv',index_col='Unnamed: 0',parse_dates=True)

ValueError: 'Unnamed: 0' is not in list

In [ ]:
dataset.head()

In [ ]:
dataset["zelma"] = zlema(dataset['Close'],length=5)

/tmp/ipython-input-4-405075763.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ema_data = src + (src - src[lag])


In [ ]:
dataset.to_csv('/content/drive/MyDrive/KFUPM/Graduation/data/US/BAC_selected_features.csv', index= False)

In [ ]:
company, company_selected_ti = get_company_df_us(dataset,dataset)

<ipython-input-38-97abc5262715>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return pd.DataFrame(values).fillna(method='ffill').values
/usr/local/lib/python3.11/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.11/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
<ipython-input-39-b84039fce409>:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['Close'],length=5)
<ipython-input-39-b84039fce409>:32: FutureWarning: Series.__setitem__ treating keys as positions is depre

In [ ]:
company.to_csv('/content/drive/MyDrive/KFUPM/Graduation/data/GOOG_selected_features.csv')

## Excution SA

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/KFUPM/Graduation/data/Tadawul_stcks.csv',index_col='date',parse_dates=True)
dataset.sort_values(by='date', inplace = True)

In [ ]:
selected_companies_dic = {"ADC":["ADC","2001-12-31"], "ALRAJHI":["ALRAJHI","2001-12-31"], "GACO":["GACO","2002-01-01"], "EMAAR EC":["EMAAR EC","2006-10-06"], "FITAIHI GROUP":["FITAIHI GROUP","2002-01-26"], "SARCO":["SARCO","2002-01-23"], "SAUDI ELECTRICITY":["SAUDI ELECTRICITY","2002-06-08"], "SPIMACO":["SPIMACO","2001-12-31"], "STC":["STC","2003-01-25"], "TASNEE":["TASNEE","2001-12-31"] }

In [ ]:
for company_name in selected_companies_dic:
  print(company_name)
  company, company_selected_ti = get_company_df_sa(dataset,selected_companies_dic[company_name][0],selected_companies_dic[company_name][1])
  company.to_csv(f'/content/drive/MyDrive/KFUPM/Graduation/data/{company_name}_selected_features.csv')

ADC


<ipython-input-38-97abc5262715>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return pd.DataFrame(values).fillna(method='ffill').values
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
<ipython-input-52-236bcff3115d>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input-52-236bcff3115d>:35: FutureWarning: Series.__setitem__ treating 

ALRAJHI


<ipython-input-38-97abc5262715>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return pd.DataFrame(values).fillna(method='ffill').values
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.11/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
<ipython-input-52-236bcff3115d>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser

GACO


...

<ipython-input-38-97abc5262715>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return pd.DataFrame(values).fillna(method='ffill').values
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.11/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.11/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
<ipython-input-52-236bcff3115d>:35: FutureWarning: Series.__getitem__ treating keys as positions is depreca

STC


<ipython-input-38-97abc5262715>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return pd.DataFrame(values).fillna(method='ffill').values
<ipython-input-52-236bcff3115d>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input-52-236bcff3115d>:35: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input-52-236bcff3115d>:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In

TASNEE


<ipython-input-38-97abc5262715>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return pd.DataFrame(values).fillna(method='ffill').values
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.11/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.11/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
<ipython-input-52-236bcff3115d>:35: FutureWarning: Series.__getitem__ treating keys as positions is depreca